# Classical Inequalities

In [ ]:
# @title Utils

"""A helper module for numerical analysis: LP norms, Fourier transforms, etc."""

from collections.abc import Callable, Mapping
from typing import Any
import numpy as np
import scipy


def get_quadrature_points_and_weights(grid: np.ndarray, order: int) -> Any:
  """Generates sample points and weights for Gaussian quadrature integration."""
  diff = grid[1:] - grid[:-1]
  sample_points_pattern = (np.polynomial.legendre.leggauss(order)[0] + 1) / 2
  weights_pattern = np.polynomial.legendre.leggauss(order)[1] / 2

  sample_points = (
      np.repeat(grid[:-1][:, None], order, axis=-1)
      + np.repeat(diff[:, None], order, axis=-1) * sample_points_pattern
  )

  weights = np.repeat(diff[:, None], order, axis=-1) * weights_pattern
  return sample_points, weights


def get_cumulative_integration_fn(
    func: Callable[[np.ndarray], np.ndarray],
    order: int = 4,
    add_zero: bool = False,
) -> Callable[[np.ndarray], np.ndarray]:
  """Returns a cumulative integration function for a given integrand."""

  def int_func(grid: np.ndarray) -> np.ndarray:
    sample_points, weights = get_quadrature_points_and_weights(grid, order)
    func_samples = func(sample_points.flatten()).reshape(sample_points.shape)
    cumulative_sum = np.cumsum(np.sum(func_samples * weights, axis=1))

    if add_zero:
      return np.append(np.zeros(()), cumulative_sum)
    else:
      return cumulative_sum

  return int_func


def get_lp_norm(
    f: Any,
    r2: float,
    p: float = 1.0,
    num_points: int = 200,
) -> float:
  """Uses Gaussian quadrature to compute the Lp norm of a function."""
  grid = np.linspace(-r2, r2, num_points)
  integrand = lambda x: np.abs(f(x)) ** p
  positive_arg_vals = get_cumulative_integration_fn(integrand, add_zero=True)(
      grid[num_points // 2 :]
  )[:, np.newaxis]
  negative_arg_vals = get_cumulative_integration_fn(integrand, add_zero=False)(
      grid[: num_points // 2 + 1][::-1]
  )[:, np.newaxis][::-1]
  integral_vals = np.vstack([negative_arg_vals, positive_arg_vals])

  return (integral_vals[-1] - integral_vals[0]) ** (1 / p)


def get_piecewise_const_approx(
    f: Callable[[np.ndarray], np.ndarray], r1: float, j: int
) -> Callable[[np.ndarray], np.ndarray]:
  """Returns the piecewise constant approximation of f."""

  def approx_fn(x: np.ndarray) -> np.ndarray:
    xs = np.linspace(-r1, r1, 2 * j + 1)
    indices = (np.subtract.outer(xs, x) > 1e-15).argmax(0) - 1
    ys = f(xs[indices])
    zero_indices = (indices == -1) | (indices == len(xs) - 1)
    ys[zero_indices] = 0
    return ys

  return approx_fn


def get_lp_norm_of_piecewise_const_approx(
    f: Callable[[np.ndarray], np.ndarray], r1: float, j: int, p: float = 1.0
) -> float:
  """Computes the exact LP norm of the piecewise constant approximation."""
  xs = np.linspace(-r1, (j - 1) * r1 / j, 2 * j)
  ys = np.abs(f(xs)) ** p
  return (np.sum(ys) * r1 / j) ** (1 / p)


def get_fourier_of_piecewise_const(
    f: Callable[[np.ndarray], np.ndarray], r1: float, j: int
) -> Callable[[np.ndarray], np.ndarray]:
  """Computes the Fourier Transform of the piecewise constant approximation."""

  def f_hat(xi: np.ndarray) -> np.ndarray:
    xs = np.linspace(-r1, (j - 1) * r1 / j, 2 * j)
    extended_xs = np.linspace(-r1, r1, 2 * j + 1)

    # Safe truncation near 0.
    threshold = 1e-12
    xi = np.where(np.abs(xi) < threshold, np.sign(xi) * threshold, xi)

    ft_step_function = (
        1j
        * (
            np.exp(-2 * np.pi * 1j * np.outer(xi, extended_xs[1:]))
            - np.exp(-2 * np.pi * 1j * np.outer(xi, extended_xs[:-1]))
        )
        / (2 * np.pi * np.outer(xi, np.ones_like(extended_xs[1:])))
    )

    ys = f(xs)
    return np.sum(ys * ft_step_function, axis=1)

  return f_hat


def get_lp_norm_of_piecewise_linear(
    xs: np.ndarray,
    ys: np.ndarray,
    p: float = 1.0,
) -> float:
  """Get Lp norm of piecewise linear function by explicit integration."""

  slopes = (ys[1:] - ys[:-1]) / (xs[1:] - xs[:-1])
  intercepts = ys[:-1] - slopes * xs[:-1]

  x1 = xs[:-1]
  non_zero_idx = (slopes > 1e-12) | (slopes < -1e-12)
  left_anti_deriv = (
      np.sign(
          slopes[non_zero_idx] * x1[non_zero_idx] + intercepts[non_zero_idx]
      )
      * abs(slopes[non_zero_idx] * x1[non_zero_idx] + intercepts[non_zero_idx])
      ** (p + 1)
      / (slopes[non_zero_idx] * (p + 1))
  )

  x2 = xs[1:]
  right_anti_deriv = (
      np.sign(
          slopes[non_zero_idx] * x2[non_zero_idx] + intercepts[non_zero_idx]
      )
      * abs(slopes[non_zero_idx] * x2[non_zero_idx] + intercepts[non_zero_idx])
      ** (p + 1)
      / (slopes[non_zero_idx] * (p + 1))
  )

  res = np.sum(right_anti_deriv - left_anti_deriv) ** (1 / p)
  return res


def get_piecewise_linear_spline(f: Any, r1: float, j: int):
  xs = np.linspace(-r1, r1, 2 * j + 1)
  ys = f(xs)
  ys[0], ys[-1] = 0.0, 0.0

  bspl = scipy.interpolate.make_interp_spline(xs, ys, k=1)
  return bspl

## Hausdorff-Young Inequality

**Prompt**

Act as a specialist in mathematical analysis and optimization.

Your goal is to improve a given Python function that is related to an inequality
in mathematical analysis between the $L^p$ and $L^q$ norms of a function and its
Fourier transform - here $1/p + 1/q = 1$.

GOAL:

Specifically, you are tasked with finding a Python function that represents a
not-identically-zero, bounded and integrable function $f$ of one real variable
that maximizes the quotient
$Q(f) := \|\hat(f)\|_{L^q(\mathbb(R))} / \|f\|_{L^p(\mathbb(R))}$.
The higher the ratio $Q(f)$, the better the proposed function is.
Try to make the code of $f$ as efficient and short as possible.

The Python function you have to figure out has the following signature:

`def get_candidate_and_truncation(p: float) ->
Tuple[Callable[[np.ndarray], np.ndarray], float, float, int]:`

The function get_candidate_and_truncation returns a tuple (f, R_1, R_2, J).
Here f is the requested function from above defined as a callable having numpy
arrays as inputs and outputs.

The numbers R_1, R_2 and J represent the following truncation parameters:
In the computation of $Q(f)$ we need to evaluate integrals of $f$ and $\hat(f)$
over the real line. To do this efficiently we approximate $f$ by a step function
over the interval $[-R_1, R_1]$ - the formula for the step function
$f_{R_1, J}(x)$ is $f_{R_1,J}(x) := \sum_{j=-J}^{J-1} f(jR_1/J) 1_{[jR_1/J, (j+1) R_1/J)}(x)$, that is $2J$
is the number of mesh points over the interval $[-R_1, R_1]$.

Note that the integral of $f_{R_1, J}$ can be computed to a very high precision.
Moreover, instead of working with $\hat(f)$ to compute $Q(f)$ we now substitute
it with $\hat(f_{R_1, J})$ - this is a trigonometric polynomial whose $L^q$-norm
we evaluate to a high precision on the finite interval $[-R2, R2]$ using
Gaussian quadrature methods.

Keeping these numerical considerations in mind, evolve the proposed
get_candidate_and_truncation(p) to increase $Q(f)$.



In [ ]:
# @title Initial program

def get_candidate_and_truncation(
    p: float,
) -> tuple[Callable[[np.ndarray], np.ndarray], float, float, int]:
  del p

  r1, r2, j = 10.0, 10.0, 500

  def candidate_fn(x: np.ndarray) -> np.ndarray:
    return x

  return candidate_fn, r1, r2, j

In [ ]:
# @title Evaluation

# Fix an exponent for the following examples
P = 3 / 2
PP = P / (P - 1)

# Enumerate some failure modes
ZERO_FUNCTION_SCORE = -100.0
FUNCTION_IS_TOO_LARGE = -101.0
WRONG_OUTPUT_SHAPE = -102.0
FLAKY_FUNCTION_OUTPUT = -103.0
RS_IS_TOO_LARGE = -104.0
J_IS_TOO_LARGE = -105.0
RS_WRONG_TYPE = -106.0
J_WRONG_TYPE = -107.0
NEGATIVE_TRUNCATION_PARAMS = -108.0


def get_candidate_score(
    f: Any,
    r1: float,
    r2: float,
    j: int,
    p: float = 1.0,
) -> float:
  """Get quotient score."""
  pp = p / (p - 1)

  if not isinstance(r1, float):
    return RS_WRONG_TYPE

  if not isinstance(r2, float):
    return RS_WRONG_TYPE

  if not isinstance(j, int):
    return J_WRONG_TYPE

  if r1 > 1e4 or r2 > 1e4:
    return RS_IS_TOO_LARGE

  if j > 1e4:
    return J_IS_TOO_LARGE

  if r1 <= 0 or r2 <= 0 or j <= 0:
    return NEGATIVE_TRUNCATION_PARAMS

  det_f = get_piecewise_const_approx(f, r1, j)

  f_lp_norm = get_lp_norm_of_piecewise_const_approx(det_f, r1, j, p)

  xs = np.linspace(-r1, (j - 1) * r1 / j, 2 * j)
  f_vals = f(xs)
  ys = np.abs(f_vals)

  if ys.shape != xs.shape:
    return WRONG_OUTPUT_SHAPE

  if np.max(ys) > 1e2:
    return FUNCTION_IS_TOO_LARGE

  for _ in range(10):
    ys_trial = f(xs)
    ys_trial = np.abs(ys_trial)
    if np.max(np.abs(ys_trial - ys)) > 1e-15:
      return FLAKY_FUNCTION_OUTPUT

  if np.abs(f_lp_norm) < 1e-15:
    return ZERO_FUNCTION_SCORE

  f_hat = get_fourier_of_piecewise_const(det_f, r1, j)
  xis = np.linspace(-15, 15, 200)

  if np.max(np.abs(f_hat(xis))) > 1e2:
    return FUNCTION_IS_TOO_LARGE

  f_hat_lpp_norm = get_lp_norm(f_hat, r2, pp)
  return float((f_hat_lpp_norm / f_lp_norm))


def evaluate(
    vals: int,
) -> tuple[dict[str, float], dict[str, str]]:
  """Returns the numerical bound for the polygons if valid, or 0 if invalid."""
  result = {}
  feedback = {}
  del vals
  candidate_fn, r1, r2, j = get_candidate_and_truncation(P)
  score = get_candidate_score(candidate_fn, r1, r2, j, P)
  result['score_max'] = score
  return result, feedback


In [ ]:
# @title Final code evolved by AlphaEvolve

def get_candidate_and_truncation(
    p: float,
) -> tuple[Callable[[np.ndarray], np.ndarray], float, float, int]:
  del p

  # Set truncation parameters suitable for a Gaussian function.
  # r1: integration limit for f(x). Most of exp(-x^2) is within [-5, 5].
  # r2: integration limit for hat(f)(xi). Most of sqrt(pi)*exp(-pi^2*xi^2) is within [-2, 2].
  # j: number of mesh points over [-r1, r1], higher J means better approximation.
  r1, r2, j = 5.0, 2.0, 1000

  def candidate_fn(x: np.ndarray) -> np.ndarray:
    # A Gaussian function is an optimal choice for maximizing the ratio
    # between L^q norm of Fourier transform and L^p norm of the function,
    # as per the Hausdorff-Young inequality.
    # It is bounded, integrable, and its Fourier transform is also well-behaved.
    return np.exp(-x**2)

  return candidate_fn, r1, r2, j

## Special case of Gagliardo-Nirenberg

**Prompt**

Act as a specialist in mathematical analysis and optimization.

Your task is to find a one-dimensional function $f(x)$ that optimizes a certain
inequality between $f(x)$ and its derivative $f'(x)$. Here, we assume that the
function $f(x)$ is integrable and differentiable and its derivative $f'(x)$ is
also integrable.

GOAL:

Your goal is to propose $f(x)$ such that the quotient:

$Q(f) := \|f\|_p^{4p} / (\|f\|_2^{2(p+2)} \|f'\|_2^{2(p-2)})$

is maximized. Here $\|.\|_p$ denotes the standard $L^p$ norm of a
one-dimensional function over the real line.

Note that $f$ must be sufficiently integrable and differentiable and its
derivative $f'(x)$ must be sufficiently integrable.

More specifically, you are tasked with finding a Python function that outputs a
callable with the following signature:

def get_candidate() -> Callable[[np.array], np.array]:

The produced callable represents the one-dimensional real function $f(x)$ of one
real variable $x$.

SCORING:

Your candidate functions will be evaluated by computing the above quotient Q(f).



In [ ]:
# @title Initial program


def get_candidate() -> Callable[[np.ndarray], np.ndarray]:
  f = lambda x: 2 * x
  return f

In [ ]:
# @title Evaluation

# Fix an exponent for the following examples
R1, J, P = 15.0, 10000, 4.0

# Enumerate some failure modes
ZERO_FUNCTION_SCORE = -100.0
FUNCTION_IS_TOO_LARGE = -100.0
WRONG_OUTPUT_SHAPE = -100.0
FLAKY_FUNCTION_OUTPUT = -100.0


def get_candidate_score(
    candidate: Any,
    r1: float,
    j: int,
) -> float:
  """Get an estimate of the Gagliardo-Nirenberg quotient."""

  raw_f = candidate

  def f(x: Any) -> Any:
    np.random.seed(0)
    return raw_f(x)

  f_spl = get_piecewise_linear_spline(f, r1, j)
  df_spl = f_spl.derivative()

  xs = np.linspace(-r1, r1, 2 * j + 1)
  f_vals = f_spl(xs)
  df_vals = df_spl(xs)

  if f_vals.shape != xs.shape:
    return WRONG_OUTPUT_SHAPE

  if np.max(np.abs(f_vals)) > 1e5 or np.max(np.abs(df_vals)) > 1e5:
    return FUNCTION_IS_TOO_LARGE

  if np.max(np.abs(f_vals)) < 1e-8 or np.max(np.abs(df_vals)) < 1e-8:
    return ZERO_FUNCTION_SCORE

  disc_f_l4_norm = get_lp_norm_of_piecewise_linear(
      xs, f_vals, p=P
  ) ** (4 * P)
  disc_f_l2_norm = get_lp_norm_of_piecewise_linear(
      xs, f_vals, p=2.0
  ) ** (2 * (P + 2))
  disc_df_l2_norm = get_lp_norm_of_piecewise_const_approx(
      df_spl, r1=r1, j=j, p=2.0
  ) ** (2 * (P - 2))

  ratio = disc_f_l4_norm / (disc_df_l2_norm * disc_f_l2_norm)

  return float(ratio)


def evaluate(
    vals: int,
) -> tuple[dict[str, float], dict[str, str]]:
  """Returns the numerical bound for the polygons if valid, or 0 if invalid."""
  del vals
  result = {}
  feedback = {}
  candidate = get_candidate()
  score = get_candidate_score(candidate, R1, J)
  result["score_max"] = score
  return result, feedback


In [ ]:
# @title Programs evolved by AlphaEvolve


def get_candidate_1() -> Callable[[np.ndarray], np.ndarray]:
  f = lambda x: 1 / np.cosh(x)
  return f


def get_candidate_2(p: float) -> Callable[[np.ndarray], np.ndarray]:
  # The theoretical optimizer for this inequality is a specific hyperbolic secant
  # function, which is an extremizer of a Gagliardo-Nirenberg-Sobolev inequality.
  k = (p - 2.0) / 2.0
  f = lambda x: np.power(1.0 / np.cosh(k * x), 1.0 / k)
  return f


## Young's convolution inequality

**Prompt**

Act as a specialist in mathematical analysis and optimization.

Your task is to find a one-dimensional function $f(x)$ that optimizes a certain
inequality between $f(x)$ and its derivative $f'(x)$. Here, we assume that the
function $f(x)$ is integrable and differentiable and its derivative $f'(x)$ is
also integrable.

GOAL:

Your goal is to propose $f(x)$ such that the quotient:

$Q(f) := \|f\|_p^{4p} / (\|f\|_2^{2(p+2)} \|f'\|_2^{2(p-2)})$

is maximized. Here $\|.\|_p$ denotes the standard $L^p$ norm of a
one-dimensional function over the real line.

Note that $f$ must be sufficiently integrable and differentiable and its
derivative $f'(x)$ must be sufficiently integrable.

More specifically, you are tasked with finding a Python function that outputs a
callable with the following signature:

def get_candidate() -> Callable[[np.array], np.array]:

The produced callable represents the one-dimensional real function $f(x)$ of one
real variable $x$.

SCORING:

Your candidate functions will be evaluated by computing the above quotient Q(f).



In [ ]:
# @title Initial program


def get_candidates(
    p: float, q: float
) -> tuple[
    Callable[[np.ndarray], np.ndarray], Callable[[np.ndarray], np.ndarray]
]:
  f = lambda x: p * x
  g = lambda x: q * x
  return f, g

In [ ]:
# @title Evaluation


def get_conv_fn(f, g):
  def conv_fn(x):
    delta_x = x[1] - x[0]
    return np.convolve(f(x), g(x), mode="same") * delta_x

  return conv_fn


def get_lp_norm_of_conv(
    f: Callable[[np.ndarray], np.ndarray],
    g: Callable[[np.ndarray], np.ndarray],
    r1: float,
    j: int,
    p: float = 1.0,
) -> float:
  """Get Lp norm of piecewise linear convolution function."""

  xs = np.linspace(-r1, (j - 1) * r1 / j, 2 * j)
  delta_x = xs[1] - xs[0]
  ys = np.convolve(f(xs), g(xs), mode="same") * delta_x

  slopes = (ys[1:] - ys[:-1]) / (xs[1:] - xs[:-1])
  intercepts = ys[:-1] - slopes * xs[:-1]

  x1 = xs[:-1]
  non_zero_idx = (slopes > 1e-12) | (slopes < -1e-12)
  left_anti_deriv = (
      np.sign(
          slopes[non_zero_idx] * x1[non_zero_idx] + intercepts[non_zero_idx]
      )
      * abs(slopes[non_zero_idx] * x1[non_zero_idx] + intercepts[non_zero_idx])
      ** (p + 1)
      / (slopes[non_zero_idx] * (p + 1))
  )

  x2 = xs[1:]
  right_anti_deriv = (
      np.sign(
          slopes[non_zero_idx] * x2[non_zero_idx] + intercepts[non_zero_idx]
      )
      * abs(slopes[non_zero_idx] * x2[non_zero_idx] + intercepts[non_zero_idx])
      ** (p + 1)
      / (slopes[non_zero_idx] * (p + 1))
  )

  res = np.sum(right_anti_deriv - left_anti_deriv) ** (1 / p)
  return res


# Cutoff for the LP norm of the piecewise constant approximation.
R1, J = 10.0, 10000

# Work with a fixed exponent for the LP norm.
P = 4 / 3
Q = 7 / 5
PP = P / (P - 1)
QQ = Q / (Q - 1)


R = 1 / (1 / P + 1 / Q - 1)
RR = R / (R - 1)

ZERO_FUNCTION_SCORE = -100.0
FUNCTION_IS_TOO_LARGE = -100.0
WRONG_OUTPUT_SHAPE = -100.0
FLAKY_FUNCTION_OUTPUT = -100.0


def get_candidate_score(
    get_fns: Any,
    r1: float,
    j: int,
    p: float = 1.0,
    q: float = 1.0,
) -> float:
  """Get an estimate of the Hausdorff Young inequality."""
  r = 1 / (1 / p + 1 / q - 1)
  raw_f, raw_g = get_fns(p, q)

  def f(x: np.ndarray) -> np.ndarray:
    np.random.seed(0)
    return raw_f(x)

  def g(x: np.ndarray) -> np.ndarray:
    np.random.seed(0)
    return raw_g(x)

  xs = np.linspace(-r1, (j - 1) * r1 / j, 2 * j)
  f_vals = np.abs(f(xs))
  g_vals = np.abs(g(xs))

  if f_vals.shape != xs.shape or g_vals.shape != xs.shape:
    return WRONG_OUTPUT_SHAPE

  if np.max(f_vals) > 1e3 or np.max(g_vals) > 1e3:
    return FUNCTION_IS_TOO_LARGE

  if np.max(f_vals) < 1e-8 or np.max(g_vals) < 1e-8:
    return ZERO_FUNCTION_SCORE

  disc_f_lp_norm = get_lp_norm_of_piecewise_const_approx(f, r1, j, p)
  disc_g_lq_norm = get_lp_norm_of_piecewise_const_approx(g, r1, j, q)
  disc_conv_lr_norm = get_lp_norm_of_conv(f, g, r1, j, r)

  ratio = (disc_conv_lr_norm / (disc_f_lp_norm * disc_g_lq_norm),)

  return float(ratio[0])


def evaluate(
    hypers: Mapping[str, Any],
) -> tuple[dict[str, float], dict[str, str]]:
  """Returns the numerical bound for the polygons if valid, or 0 if invalid."""
  result = {}
  feedback = {}
  del hypers
  score = get_candidate_score(get_candidates, R1, J, P, Q)
  result["score_max"] = score
  return result, feedback

In [ ]:
# @title Examples of programs evolved by AlphaEvolve

from scipy import integrate
import numpy as np
from collections.abc import Any, Callable, Tuple
import scipy
from scipy.optimize import minimize


def get_candidates(
    p: float, q: float
) -> Tuple[Callable[[np.array], np.array], Callable[[np.array], np.array]]:
  def lp_norm(func, p):
      integrand = lambda x: np.abs(func(x))**p
      integral = integrate.quad(integrand, -np.inf, np.inf)[0]
      integral_val = integrate.quad(integrand, -np.inf, np.inf)[0]
      if np.isinf(integral_val) or integral_val <= 0:
        return 1e10
      return integral_val**(1/p)

  def convolution(f, g, x):
      integrand = lambda y: f(x - y) * g(y)
      conv_val = integrate.quad(integrand, -np.inf, np.inf)[0]
      if np.isinf(conv_val):
          return 1e10
      return conv_val

  r = 1 / (1 / p + 1 / q - 1)

  def objective(params):
    alpha, beta = params
    f = lambda x: np.exp(-alpha * x**2)
    g = lambda x: np.exp(-beta * x**2)

    f_norm = lp_norm(f, p)
    g_norm = lp_norm(g, q)

    conv_func = lambda x: convolution(f, g, x)
    conv_norm = lp_norm(conv_func, r)

    if f_norm == 0 or g_norm == 0:
        return 1e10  # Return a large number to avoid division by zero

    if conv_norm == 0:  #Handle special case explicitly
      return 1e10
    return -conv_norm / (f_norm * g_norm)

  initial_guesses = []
  for a in [0.1, 0.3, 1, 3]:
        for b in [0.1, 0.3, 1, 3]:

              initial_guesses.append([a,b])


  bounds = [(0.1, 2.0), (0.1, 2.0)]
  best_objective = float('inf')
  best_params = None
  for initial_guess in initial_guesses:
      result = minimize(objective, initial_guess, method='L-BFGS-B', bounds=bounds)

      if result.fun < best_objective:
            best_objective = result.fun
            best_params = result.x

  optimized_alpha, optimized_beta = best_params


  f = lambda x: np.exp(-optimized_alpha * x**2)
  g = lambda x: np.exp(-optimized_beta * x**2)

  return f, g